In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import datetime
import os
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from matplotlib import pyplot
from pylab import rcParams
import pickle as pk
import gc
import requests 
import matplotlib.gridspec as gridspec
import netCDF4 as nc

In [ ]:
with open('/home/jovyan/data/botpt/bottom_pressure15S_F11.pkl', 'rb') as E:
    botpt_data = pk.load(E)
df_botpt = pd.DataFrame(botpt_data)
df_botpt['bottom_pressure'] = df_botpt['bottom_pressure'].astype(float)
df_botpt['depth']=df_botpt['bottom_pressure'].astype(float) * 0.670
#MJ03F_cal_depths = [MJ03F_pressure * 0.0670 for MJ03F_pressure in MJ03F_pressure]
#list comprehention
epoch= [i.timestamp() for i in df_botpt.index.to_pydatetime()]
df_botpt['epoch'] = epoch
df_botpt= df_botpt.sort_index()

In [ ]:
with open('/home/jovyan/data/botpt/bottom_pressure15S_E11.pkl', 'rb') as E:
    botpt_data = pk.load(E)
df_botptE = pd.DataFrame(botpt_data)
df_botptE['bottom_pressure'] = df_botptE['bottom_pressure'].astype(float)
df_botptE['depth']=df_botptE['bottom_pressure'].astype(float) * 0.670
#MJ03F_cal_depths = [MJ03F_pressure * 0.0670 for MJ03F_pressure in MJ03F_pressure]
#list comprehention
epoch= [i.timestamp() for i in df_botptE.index.to_pydatetime()]
df_botptE['epoch'] = epoch
df_botptE= df_botptE.sort_index()

#### Cleaning up the DataFrame

In [ ]:
del df_botptE['epoch']
del df_botptE['bottom_pressure']
del df_botpt['epoch']
del df_botpt['bottom_pressure']

In [ ]:
df_botpt.tail()

In [ ]:
df_botptE.head()

#### Merge BOTPT E and BOTPT F 

In [ ]:
test = pd.merge(df_botpt, df_botptE,how='outer', indicator=True, left_index=True, right_index=True, suffixes=('_F', '_E'))

In [ ]:
df_botptMerge = test[test['_merge'] == 'both']
del df_botptMerge['_merge']

In [ ]:
df_botptMerge = df_botptMerge.loc['2016-12-1 00:00:00':'2019-06-17 00:00:00']

In [ ]:
df_botptMerge.tail()

#### Calculate Depth difference 

In [ ]:
depthDiff = df_botptMerge['depth_E'].values - df_botptMerge['depth_F'].values

In [ ]:
df_botptMerge['diff'] = depthDiff

#### Create time and height vectors for plotting 

In [ ]:
# time = list(df_botptMerge.index.values)
#height = x.tolist()
height = df_botptMerge['diff'].tolist()
time_int = []
time = list(pd.to_datetime(df_botptMerge.index.values))
for i in time:
    i = np.datetime64(i).astype(datetime.datetime)
    time_int.append(dates.date2num(i))

In [ ]:
plt.close()
fig, ax = plt.subplots()
fig.set_size_inches(28, 6)
hb1 = ax.hexbin(time_int, height, vmin=0, vmax=30, gridsize=(1500,100), mincnt=1, cmap='Greens', linewidths=0)
fig.colorbar(hb1, pad = 0.01)
ax.yaxis.grid(True)
ax.xaxis.grid(True)
ax.set_xlim(datetime.datetime(2017, 1, 1, 0, 0),datetime.datetime(2018, 10, 31, 0, 0))
years = dates.YearLocator()
months = dates.MonthLocator()
yearsFmt = dates.DateFormatter('\n\n\n%Y')
monthsFmt = dates.DateFormatter('%b')
ax.xaxis.set_major_locator(months)
ax.xaxis.set_major_formatter(monthsFmt)
ax.xaxis.set_minor_locator(years)
ax.xaxis.set_minor_formatter(yearsFmt)
plt.tight_layout()
plt.setp(ax.xaxis.get_majorticklabels(), rotation=90)
#plt.gca().invert_yaxis()
plt.show()

In [ ]:
# plt.savefig('Figures/2015thruPresent.png')

In [ ]:
df_botptMerge['date']=pd.DatetimeIndex(df_botptMerge.index).date
#df_botptMerge.head()

#### Use Groupby to create one day mean measurements

In [ ]:
df_botptMean=df_botptMerge.groupby('date').mean()
df_botptMean.head()

#### Create time and height vectors for plotting 

In [ ]:
# time = list(df_botptMerge.index.values)
#height = x.tolist()
height = df_botptMean['diff'].tolist()
time_int = []
time = list(pd.to_datetime(df_botptMean.index.values))
for i in time:
    i = np.datetime64(i).astype(datetime.datetime)
    time_int.append(dates.date2num(i))

In [ ]:
plt.close()
fig, ax = plt.subplots()
fig.set_size_inches(28, 6)
hb1 = ax.plot(time_int, height)
ax.yaxis.grid(True)
ax.xaxis.grid(True)
ax.set_xlim(datetime.datetime(2017, 1, 1, 0, 0),datetime.datetime(2018, 10, 31, 0, 0))
years = dates.YearLocator()
months = dates.MonthLocator()
yearsFmt = dates.DateFormatter('\n\n\n%Y')
monthsFmt = dates.DateFormatter('%b')
ax.xaxis.set_major_locator(months)
ax.xaxis.set_major_formatter(monthsFmt)
ax.xaxis.set_minor_locator(years)
ax.xaxis.set_minor_formatter(yearsFmt)
plt.tight_layout()
plt.setp(ax.xaxis.get_majorticklabels(), rotation=90)
#plt.gca().invert_yaxis()
plt.show()

Haley Cabaniss with significant help from Bill Chadwick, Zach Billings, Sage Lichtenwainer, and Liana Vaccari

#### Bill Chadwick recently noticed spikes in BOTPT data for 3 of the instruments on the cabled array. He notified Dana Manalung, who attributed the spikes to a grounding problem in the junction boxes. During a recent cruise (early July 2018) the junction boxes were changed and the 3 BOTPT instruments (MJ03B, MJ03B, MJ03F) were grounded using an alternatitve method. 

## Remove Spikes 

In [ ]:
weird_data = np.array(height)
time_array = np.array(time)

cleaner_data = []
cleaner_time = []

# i = 0
# while i < len(weird_data):
#     if (weird_data[i] - weird_data[i-1] > 2

length = len(weird_data)

i = 0
while i < length:
    j = i+2
    while j < length:
        if np.abs(weird_data[i] - weird_data[j]) < .05:
            cleaner_data.append(weird_data[j])
            cleaner_time.append(time_array[j])
            break
        else:
            j += 1
    i = j


### Comparison between plots with and without spikes. 

In [ ]:
# plt.savefig('Figures/2018.png')

In [ ]:
plt.close()
fig3, (ax1,ax2) = plt.subplots(2,1)
fig3.set_size_inches(28, 14)
hb1 = ax1.plot(time_int, height, linewidth=5)
ax1.yaxis.grid(True)
ax1.xaxis.grid(True)
ax1.set_xlim(datetime.datetime(2017, 1, 1, 0, 0),datetime.datetime(2018, 10, 31, 0, 0))
years = dates.YearLocator()
months = dates.MonthLocator()
yearsFmt = dates.DateFormatter('\n\n\n%Y')
monthsFmt = dates.DateFormatter('%b')
ax1.xaxis.set_major_locator(months)
ax1.xaxis.set_major_formatter(monthsFmt)
ax1.xaxis.set_minor_locator(years)
ax1.xaxis.set_minor_formatter(yearsFmt)

hb1 = ax2.plot(cleaner_time, cleaner_data, linewidth=5)
ax2.yaxis.grid(True)
ax2.xaxis.grid(True)
ax2.set_xlim(datetime.datetime(2017, 1, 1, 0, 0),datetime.datetime(2018, 10, 31, 0, 0))
years = dates.YearLocator()
months = dates.MonthLocator()
yearsFmt = dates.DateFormatter('\n\n\n%Y')
monthsFmt = dates.DateFormatter('%b')
ax2.xaxis.set_major_locator(months)
ax2.xaxis.set_major_formatter(monthsFmt)
ax2.xaxis.set_minor_locator(years)
ax2.xaxis.set_minor_formatter(yearsFmt)

plt.tight_layout()
plt.setp(ax1.xaxis.get_majorticklabels(), rotation=90)
plt.suptitle('Removing Outliers', fontsize=32, color= 'red', fontweight = 'bold')
plt.subplots_adjust(top=0.90)
#plt.gca().invert_yaxis()
plt.show()

In [ ]:
# plt.savefig('Figures/2018.png')

#### Import Seismic Data 

In [ ]:
!tail /home/jovyan/data/hypo71.dat.txt

#### Import Seismic Data 

In [ ]:
seismic_file = '/home/jovyan/data/hypo71.dat.txt'
df_seismic_data = pd.read_csv(seismic_file, delim_whitespace=True, dtype=object)
df_seismic_data['datetime'] = df_seismic_data['yyyymmdd'] + 'T' + \
            df_seismic_data['HHMM'].str.slice(start=0, stop=2) + ':' + \
            df_seismic_data['HHMM'].str.slice(start=2) 
df_seismic_data.index = pd.to_datetime(df_seismic_data['datetime'].values)
df_seismic_data['datetime'] = pd.to_datetime(df_seismic_data['datetime'].values)
df_seismic_data = df_seismic_data.loc['2017-01-1 00:00:00':'2019-06-17 00:00:00']
df_seismic_data.head()
# del df_seismic_data['yyyymmdd']
# del df_seismic_data['HHMM']
del df_seismic_data['Lon(D']
# del df_seismic_data['SSS.SS']
# del df_seismic_data['Depth']
del df_seismic_data['M)']
del df_seismic_data['M).1']
del df_seismic_data['NWR']
del df_seismic_data['GAP']
del df_seismic_data['DMIN']
del df_seismic_data['ERH']
del df_seismic_data['ERZ']
del df_seismic_data['ID']
del df_seismic_data['Lat(D']
del df_seismic_data['PMom']
del df_seismic_data['SMom']
df_seismic_data['Depth'] = df_seismic_data['Depth'].astype('float64').values
df_seismic_data['MW'] = df_seismic_data['MW'].astype('float64').values
df_seismic_data['RMS'] = df_seismic_data['RMS'].astype('float64').values

In [ ]:
df_seismic_data.datetime.astype(np.int64).values/1e64
df_seismic_data['date'] =pd.DatetimeIndex(df_seismic_data.datetime).date
df_seismic_data.tail()

In [ ]:
df_seismic_data.plot(x='datetime', y= 'Depth')

In [ ]:
df_seismic_data.plot(x='datetime', y= 'MW')

In [ ]:
df_seismic_data.plot(x='datetime', y= 'RMS')

In [ ]:
df_seismic_data.datetime.astype(np.int64).values/1e64
df_seismic_data['date'] =pd.DatetimeIndex(df_seismic_data.datetime).date
df_seismic_data.tail()

#### Create dataframe with Earthquake frequency (count per day)
##### Note: Days with zero earthquakes are not represented in this timeseries. 

In [ ]:
df_eqMean=df_seismic_data.groupby('date').mean()
df_eqCount= df_seismic_data.groupby('date').count()
#del df_eqMean['datetime']
#df_eqMean.columns.name = df_eqMean.index.name
#df_eqMean.index.name = None
df_eqMean.tail()

In [ ]:
df_eqCount['count'] = df_eqCount.datetime.astype('float64').values
df_eqCount.head()

In [ ]:
df_eqMean['count'] = df_eqCount['count'].values
df_eqMean.head()

#### Resample. Add days to our earthquake data where there were zero earthquakes. 

In [ ]:
idx = pd.date_range('2017-01-1 00:00:00', '2019-06-16 00:00:00')

df_eqMean = df_eqMean

df_eqMean.index = pd.DatetimeIndex(df_eqMean.index)

df_eqMean = df_eqMean.reindex(idx, fill_value=0)
df_eqMean.tail()

In [ ]:
import hvplot.pandas
a= df_eqMean.hvplot.scatter(y= 'count')
b= df_eqMean.hvplot.scatter(y= 'Depth')
a+b

In [ ]:
df_eqMean.hvplot.scatter(y= ['count', 'Depth','MW','RMS']
                        , subplots=True, shared_axes=False).cols(1)

#### Create time and count vectors earthquake frequency vectors for plotting 

In [ ]:
count = df_eqMean['count'].tolist()
time_int = []
time = list(pd.to_datetime(df_eqMean.index.values))
for i in time:
    i = np.datetime64(i).astype(datetime.datetime)
    time_int.append(dates.date2num(i))

In [ ]:
def movingaverage(interval, window_size):
    window= np.ones(int(window_size))/float(window_size)
    return np.convolve(interval, window, 'same')

In [ ]:
# smooth data using rolling window that chops off 95th percentile 
count_av = list(movingaverage(df_eqMean['count'],21))

In [ ]:
plt.close()
fig4, (ax1,ax2) = plt.subplots(2,1)
fig4.set_size_inches(30, 14)
hb1 = ax1.plot(cleaner_time, cleaner_data, linewidth=5)
ax1.yaxis.grid(True)
ax1.xaxis.grid(True)
ax1.set_xlim(datetime.datetime(2017, 1, 1, 0, 0),datetime.datetime(2019, 6, 17, 0, 0))
#years = dates.YearLocator()
months = dates.MonthLocator()
#yearsFmt = dates.DateFormatter('\n\n\n%Y')
monthsFmt = dates.DateFormatter('%b')
ax1.xaxis.set_major_locator(months)
ax1.xaxis.set_major_formatter(monthsFmt)
#ax1.xaxis.set_minor_locator(years)
#ax1.xaxis.set_minor_formatter(yearsFmt)
ax1.set_title('Caldera Inflation', fontsize=36, fontweight = 'bold')
ax1.set_ylabel('Depth-Diff (m)', fontsize=36, fontweight = 'bold', rotation=90)


hb1 = ax2.plot(time, count)
hb2 = ax2.plot(time, count_av, linewidth=5)
ax2.yaxis.grid(True)
ax2.xaxis.grid(True)
ax2.set_xlim(datetime.datetime(2017, 1, 1, 0, 0),datetime.datetime(2019, 6, 17, 0, 0))
years = dates.YearLocator()
months = dates.MonthLocator()
yearsFmt = dates.DateFormatter('\n\n\n%Y')
monthsFmt = dates.DateFormatter('%b')
ax2.xaxis.set_major_locator(months)
ax2.xaxis.set_major_formatter(monthsFmt)
ax2.xaxis.set_minor_locator(years)
ax2.xaxis.set_minor_formatter(yearsFmt)
ax2.set_title('Daily Seismicity', fontsize=36, fontweight = 'bold')
ax2.set_ylabel('EQ/Day', fontsize=36, fontweight = 'bold', rotation=90)

#plt.tight_layout()
plt.setp(ax1.set_xticklabels([]))
#plt.setp(ax1.xaxis.get_majorticklabels(), rotation=0, fontsize=24)
#plt.setp(ax1.xaxis.get_minorticklabels(), rotation=0, fontsize=24)
plt.setp(ax1.yaxis.get_majorticklabels(), rotation=0, fontsize=24, fontweight = 'bold')

plt.setp(ax2.xaxis.get_majorticklabels(), rotation=0, fontsize=24, fontweight = 'bold')
plt.setp(ax2.xaxis.get_minorticklabels(), rotation=0, fontsize=36, fontweight = 'bold')
plt.setp(ax2.yaxis.get_majorticklabels(), rotation=0, fontsize=24, fontweight = 'bold')

#plt.suptitle('Comparison Between Inflation and Seismicity',
#             fontsize=32, color= 'blue', fontweight = 'bold')
plt.subplots_adjust(top=0.90, hspace=0.25)
plt.savefig('/home/jovyan/AGU_2018/Examples/botptvseq.png')